## **<mark>User-defined Functions</mark>**

Các hàm do người dùng định nghĩa (user-defined functions) bao gồm các hàm có giá trị vô hướng (scalar-valued functions) trả về một giá trị duy nhất và hàm có giá trị bảng (table-valued functions) trả về các hàng dữ liệu.

  

Các hàm do người dùng xác định trong SQL Server giúp bạn đơn giản hóa sự phát triển của mình bằng cách đóng gói logic nghiệp vụ phức tạp và cung cấp chúng để sử dụng lại trong mọi truy vấn.

### **<mark>Hàm có giá trị vô hướng (scalar-valued functions)</mark>**

Hàm vô hướng của SQL Server nhận một hoặc nhiều tham số và <mark>trả về một giá trị duy nhất.</mark>

Các hàm vô hướng giúp bạn đơn giản hóa mã của mình. Ví dụ: bạn có thể có một phép tính phức tạp xuất hiện trong nhiều truy vấn. Thay vì bao gồm công thức trong mọi truy vấn, bạn có thể tạo một hàm vô hướng đóng gói công thức và sử dụng nó trong mỗi truy vấn.

**<mark>Tạo một hàm vô hướng</mark>**

Để tạo một hàm vô hướng, bạn sử dụng câu lệnh <mark>CREATE FUNCTION</mark> như sau:

```
CREATE FUNCTION [schema_name.]function_name (parameter_list)
RETURNS data_type AS
BEGIN
    statements
    RETURN value
END

```

Trong cú pháp này:

- Đầu tiên, chỉ định tên của hàm sau các từ khóa CREATE FUNCTION. Tên lược đồ (schema) là tùy chọn. Nếu bạn không chỉ định nó một cách rõ ràng, SQL Server sử dụng dbo theo mặc định.
- Thứ hai, chỉ định một danh sách các tham số được bao quanh bởi dấu ngoặc đơn sau tên hàm.
- Thứ ba, chỉ định kiểu dữ liệu của giá trị trả về trong câu lệnh RETURNS.
- Cuối cùng, bao gồm một câu lệnh RETURN để trả về một giá trị bên trong phần thân của hàm.

In [ ]:
CREATE FUNCTION sales.udfNetSale(
    @quantity INT,
    @list_price DEC(10,2),
    @discount DEC(4,2)
)
RETURNS DEC(10,2)
AS
BEGIN
    RETURN @quantity * @list_price * (1 - @discount);
END;

**<mark>Gọi một hàm vô hướng</mark>**

In [ ]:
SELECT 
    sales.udfNetSale(10,100,0.1) net_sale;

In [ ]:
SELECT 
    order_id, 
    SUM(sales.udfNetSale(quantity, list_price, discount)) net_amount
FROM 
    sales.order_items
GROUP BY 
    order_id
ORDER BY
    net_amount DESC;

**<mark>Sửa đổi một hàm vô hướng</mark>**

Để sửa đổi một hàm vô hướng, bạn sử dụng <mark>ALTER</mark> thay vì từ khóa <mark>CREATE</mark>. Các câu lệnh còn lại vẫn giữ nguyên:

```
ALTER FUNCTION [schema_name.]function_name (parameter_list)
    RETURNS data_type AS
    BEGIN
        statements
        RETURN value
    END

```

Lưu ý rằng bạn có thể sử dụng câu lệnh <mark>CREATE OR ALTER</mark> để tạo một hàm do người dùng xác định nếu nó không tồn tại hoặc để sửa đổi một hàm vô hướng hiện có:

```
CREATE OR ALTER FUNCTION [schema_name.]function_name (parameter_list)
        RETURNS data_type AS
        BEGIN
            statements
            RETURN value
        END

```

**<mark>Loại bỏ một hàm vô hướng</mark>**

Để loại bỏ một hàm vô hướng hiện có, bạn sử dụng câu lệnh <mark>DROP FUNCTION:</mark>

```
DROP FUNCTION [schema_name.]function_name;

```

In [ ]:
DROP FUNCTION sales.udfNetSale;

**<mark>Notes</mark>**

Sau đây là một số rút ra chính của các hàm vô hướng:

- Các hàm vô hướng có thể được sử dụng ở hầu hết mọi nơi trong các câu lệnh T-SQL.
- Các hàm vô hướng chấp nhận một hoặc nhiều tham số nhưng chỉ trả về một giá trị, do đó, chúng phải bao gồm một câu lệnh RETURN.
- Các hàm vô hướng có thể sử dụng logic như khối IF hoặc vòng lặp WHILE.
- Các hàm vô hướng không thể cập nhật dữ liệu.
- Hàm vô hướng có thể gọi các hàm khác.

### **<mark>Hàm có giá trị bảng (table-valued functions)</mark>**

Hàm có giá trị bảng là một hàm do người dùng định nghĩa trả về dữ liệu của một loại bảng. Kiểu trả về của một hàm có giá trị bảng là một bảng, do đó, bạn có thể sử dụng hàm định trị bảng giống như cách bạn sử dụng một bảng.

**<mark>Tạo một hàm có giá trị bảng</mark>**

In [ ]:
CREATE FUNCTION udfProductInYear (
    @model_year INT
)

RETURNS TABLE
AS
RETURN
    SELECT 
        product_name,
        model_year,
        list_price
    FROM
        production.products
    WHERE
        model_year = @model_year;

**<mark>Thực thi một hàm có giá trị bảng</mark>**

In [ ]:
SELECT 
    * 
FROM 
    udfProductInYear(2017);

Bạn cũng có thể chỉ định cột nào sẽ được trả về từ hàm giá trị bảng như sau:

In [ ]:
SELECT 
    product_name,
    list_price
FROM 
    udfProductInYear(2018);

**<mark>Sửa đổi một hàm có giá trị bảng</mark>**

Để sửa đổi một hàm có giá trị bảng, bạn sử dụng ALTER thay vì từ khóa CREATE. Phần còn lại của kịch bản giống nhau.

In [ ]:
ALTER FUNCTION udfProductInYear (
    @start_year INT,
    @end_year INT
)
RETURNS TABLE
AS
RETURN
    SELECT 
        product_name,
        model_year,
        list_price
    FROM
        production.products
    WHERE
        model_year BETWEEN @start_year AND @end_year

In [ ]:
SELECT 
    product_name,
    model_year,
    list_price
FROM 
    udfProductInYear(2017,2018)
ORDER BY
    product_name;

**<mark>Các hàm có giá trị trong bảng nhiều câu lệnh (Multi-statement table-valued functions)</mark>**

Một hàm giá trị bảng nhiều câu lệnh hoặc MSTVF là một hàm giá trị bảng trả về kết quả của nhiều câu lệnh.

Hàm multi-statement-table-value rất hữu ích vì bạn có thể thực hiện nhiều truy vấn trong hàm và tổng hợp kết quả vào bảng trả về.

Để xác định một hàm giá trị bảng nhiều câu lệnh, bạn sử dụng một biến bảng làm giá trị trả về. Bên trong hàm, bạn thực hiện một hoặc nhiều truy vấn và chèn dữ liệu vào biến bảng này.

In [ ]:
CREATE FUNCTION udfContacts()
RETURNS @contacts TABLE (
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(255),
    phone VARCHAR(25),
    contact_type VARCHAR(20)
)
AS
BEGIN
    INSERT INTO @contacts
    SELECT 
        first_name, 
        last_name, 
        email, 
        phone,
        'Staff'
    FROM
        sales.staffs

    INSERT INTO @contacts
    SELECT 
        first_name, 
        last_name, 
        email, 
        phone,
        'Customer'
    FROM
        sales.customers

    RETURN
END;

In [ ]:
SELECT 
    * 
FROM
    udfContacts();

In [ ]:
CREATE OR ALTER FUNCTION udfSplit(
    @string VARCHAR(MAX), 
    @delimiter VARCHAR(50) = ' ')
RETURNS @parts TABLE
(    
    idx INT IDENTITY PRIMARY KEY,
    val VARCHAR(MAX)   
)
AS
BEGIN
    DECLARE @index INT = -1;

    WHILE (LEN(@string) > 0) 
    BEGIN 
        SET @index = CHARINDEX(@delimiter , @string)  ;
        
        IF (@index = 0) AND (LEN(@string) > 0)  
        BEGIN  
            INSERT INTO @parts 
            VALUES (@string);
            BREAK  
        END 

        IF (@index > 1)  
        BEGIN  
            INSERT INTO @parts 
            VALUES (LEFT(@string, @index - 1));
            
            SET @string = RIGHT(@string, (LEN(@string) - @index));  
        END 
        ELSE
            SET @string = RIGHT(@string, (LEN(@string) - @index)); 
    END
    RETURN
END
GO

In [ ]:
SELECT 
    * 
FROM 
    udfSplit('foo,bar,baz',',');

<mark>ý tưởng là tách chuỗi gì đó thành 3 dòng</mark>

**<mark>Các trường hợp sử dụng hàm có giá trị bảng</mark>**

Chúng tôi thường sử dụng các hàm có giá trị bảng làm dạng xem được tham số hóa (parameterized views). So với các thủ tục được lưu trữ (stored procedures), các hàm định trị bằng bảng linh hoạt hơn vì chúng ta có thể sử dụng chúng ở bất kỳ nơi nào bảng được sử dụng.

## **<mark>Xóa các hàm do người dùng định nghĩa</mark>**

Để loại bỏ một hàm do người dùng định nghĩa được tạo bởi câu lệnh CREATE FUNCTION, bạn sử dụng câu lệnh DROP FUNCTION như sau:

```
DROP FUNCTION [ IF EXISTS ] [ schema_name. ] function_name;

```

Trong cú pháp này:

- IF EXISTS: Tùy chọn IF EXISTS cho phép bạn loại bỏ chức năng chỉ khi nó tồn tại. Nếu không, câu lệnh không làm gì cả. Nếu bạn cố gắng loại bỏ một hàm không tồn tại mà không chỉ định tùy chọn IF EXISTS, bạn sẽ gặp lỗi.
- schema\_name: Schema\_name chỉ định tên của lược đồ chứa hàm do người dùng xác định mà bạn muốn xóa. Tên lược đồ là tùy chọn.
- function\_name: function\_name là tên của hàm mà bạn muốn loại bỏ.

Ghi chú

- Nếu có các ràng buộc như CHECK hoặc DEFAULT và các cột được tính toán tham chiếu đến hàm, câu lệnh DROP FUNCTION cũng sẽ không thành công.

Để loại bỏ nhiều hàm do người dùng xác định, bạn chỉ định danh sách tên hàm được phân tách bằng dấu phẩy sau mệnh đề DROP FUNCTION như sau:

```
DROP FUNCTION [IF EXISTS] 
    schema_name.function_name1, 
    schema_name.function_name2,
    ...;
```

In [ ]:
CREATE FUNCTION sales.udf_get_discount_amount (
    @quantity INT,
    @list_price DEC(10,2),
    @discount DEC(4,2) 
)
RETURNS DEC(10,2) 
AS 
BEGIN
    RETURN @quantity * @list_price * @discount
END

In [ ]:
DROP FUNCTION IF EXISTS sales.udf_get_discount_amount;